This is a notebook for testing how to interact with TAQL

In [1]:
import casacore.tables as pt
import numpy as np
import ccal 

In [2]:
#define some tables/ms for test use
basedir = '/data/adams/apertif/crosscal/181026063/00/raw/'
bptable = basedir+'WSRTA181026063_B000.Bscan'
aptable = basedir+'WSRTA181026063_B000.G1ap'
msfile = basedir+'WSRTA181026063_B000.MS'

In [5]:
apt= pt.table(aptable, readonly=True)
apt

Successful readonly open of default-locked table /data/adams/apertif/crosscal/181026063/00/raw/WSRTA181026063_B000.G1ap: 13 columns, 204 rows


TIME,FIELD_ID,SPECTRAL_WINDOW_ID,ANTENNA1,ANTENNA2,INTERVAL,SCAN_NUMBER,OBSERVATION_ID,CPARAM,PARAMERR,FLAG,SNR,WEIGHT
28-Oct-2018/23:59:31,0,0,0,0,0 s,0,0,"[[(14.306388+0j), (13.024709+0j)]]","[[0.03504033, 0.024589341]]","[[False, False]]","[[408.28348, 529.6892]]",[]
28-Oct-2018/23:59:31,0,0,1,0,0 s,0,0,"[[(11.570813-0.7623296j), (-12.141009-3.197891j)]]","[[0.034268565, 0.024525572]]","[[False, False]]","[[338.38297, 511.91888]]",[]
28-Oct-2018/23:59:31,0,0,2,0,0 s,0,0,"[[(14.05032+1.3499526j), (18.235924+2.916084j)]]","[[0.03494398, 0.025216434]]","[[False, False]]","[[403.93286, 732.36395]]",[]
28-Oct-2018/23:59:31,0,0,3,0,0 s,0,0,"[[(11.1794405+0.3443882j), (-12.36486-11.844353j)]]","[[0.034169804, 0.025023889]]","[[False, False]]","[[327.3283, 684.2443]]",[]
28-Oct-2018/23:59:31,0,0,4,0,0 s,0,0,"[[(8.737918-0.61262995j), (-10.136937-6.6149516j)]]","[[0.033675563, 0.02444119]]","[[False, False]]","[[260.1105, 495.24353]]",[]
28-Oct-2018/23:59:31,0,0,5,0,0 s,0,0,"[[(10.827539-4.3756576j), (17.456398+6.405718j)]]","[[0.03428131, 0.025225477]]","[[False, False]]","[[340.65994, 737.13556]]",[]
28-Oct-2018/23:59:31,0,0,6,0,0 s,0,0,"[[(2.5410123+0.8493689j), (-2.0020823+8.020231j)]]","[[0.03307861, 0.02419768]]","[[False, False]]","[[80.99527, 341.61722]]",[]
28-Oct-2018/23:59:31,0,0,7,0,0 s,0,0,"[[(19.342516-5.542728j), (-21.942675+14.164807j)]]","[[0.037305918, 0.026627542]]","[[False, False]]","[[539.3515, 980.84454]]",[]
28-Oct-2018/23:59:31,0,0,8,0,0 s,0,0,"[[(11.909415+0.027044402j), (20.656998+8.053022j)]]","[[0.0342065, 0.025752127]]","[[False, False]]","[[348.16324, 860.94684]]",[]
28-Oct-2018/23:59:31,0,0,9,0,0 s,0,0,"[[(12.757119+2.6027474j), (13.867816+2.4883366j)]]","[[0.034559388, 0.024596762]]","[[False, False]]","[[376.74054, 572.81085]]",[]


In [6]:
taql_antnames = "SELECT NAME FROM {0}::ANTENNA".format(aptable)
t= pt.taql(taql_antnames)
ant_names=t.getcol("NAME")
print ant_names

['RT2', 'RT3', 'RT4', 'RT5', 'RT6', 'RT7', 'RT8', 'RT9', 'RTA', 'RTB', 'RTC', 'RTD']


In [58]:
#Looking at things more closely,
#this amp gain table has single solution per antenna
#but for each time step
#So get for each antenna, and maybe plot as function of time

gain_ant_array = np.empty(len(ant_names),dtype=object)
for ant in xrange(len(ant_names)):
    taql_command = ("SELECT TIME,abs(CPARAM) AS amp, arg(CPARAM) AS phase FROM {0} " 
                    "WHERE ANTENNA1={1}").format(aptable,ant)
    t = pt.taql(taql_command)
    gain_ant_array[ant] = t.getcol('amp')
    times = t.getcol('TIME')
    
print len(gain_ant_array[0]),len(times)

print times

print gain_ant_array.shape
print gain_ant_array[0].shape
print gain_ant_array[0]

17 17
[5.04748797e+09 5.04748800e+09 5.04748803e+09 5.04748806e+09
 5.04748809e+09 5.04748812e+09 5.04748816e+09 5.04748819e+09
 5.04748822e+09 5.04748825e+09 5.04748828e+09 5.04748831e+09
 5.04748834e+09 5.04748837e+09 5.04748840e+09 5.04748843e+09
 5.04748846e+09]
(12,)
(17, 1, 2)
[[[14.3063879  13.02470875]]

 [[14.3237524  13.03087997]]

 [[14.29784966 13.02768326]]

 [[14.28405476 13.03667355]]

 [[14.28699493 13.03462696]]

 [[14.28482056 13.03209877]]

 [[14.31024742 13.04302502]]

 [[14.31948662 12.99774456]]

 [[14.30723953 13.040308  ]]

 [[14.28339577 13.03633785]]

 [[14.27083492 13.04828072]]

 [[14.27350044 13.03596497]]

 [[14.30122471 13.07475853]]

 [[14.28787804 13.04773426]]

 [[14.29699993 13.02271366]]

 [[14.31365776 13.03462791]]

 [[14.28335953 13.03341866]]]


The code above is appropriate for figuring out how to look at gain tables.


Next, I want to figure out how to plot corrected gains from MS, averaging over baselines and time

In [37]:
taql_antnames = "SELECT NAME FROM {0}::ANTENNA".format(msfile)
t= pt.taql(taql_antnames)
ant_names=t.getcol("NAME")
#print ant_names

taql_freq = "SELECT CHAN_FREQ FROM {0}::SPECTRAL_WINDOW".format(msfile)
t = pt.taql(taql_freq)
freqs = t.getcol('CHAN_FREQ')

vis_corrected_ant = np.empty(len(ant_names),dtype=object)
for ant in xrange(len(ant_names)):
    taql_command = ("SELECT CORRECTED_DATA FROM {0} " 
                    "WHERE ANTENNA1!=ANTENNA2 && "
                    "(ANTENNA1={1} || ANTENNA2={1})").format(msfile,ant)
    t = pt.taql(taql_command)
    vis_corrected_ant[ant] = t.getcol('CORRECTED_DATA')
#    times = t.getcol('TIME')
    
taql_time =  "select TIME from {0} orderby unique TIME".format(msfile)
t= pt.taql(taql_time)
times = t.getcol('TIME')    
print len(times)
    
print times

print vis_corrected_ant[0].shape,vis_corrected_ant[11].shape
#198 should be n(times)*n(baseline),nchan,npol
print len(times),len(times)*11
#print times

#why does this have extra timestamp (at end) compared to gain solutions?
#don't understand but not really necessary to continuing on

18
[5.04748797e+09 5.04748800e+09 5.04748803e+09 5.04748806e+09
 5.04748809e+09 5.04748812e+09 5.04748816e+09 5.04748819e+09
 5.04748822e+09 5.04748825e+09 5.04748828e+09 5.04748831e+09
 5.04748834e+09 5.04748837e+09 5.04748840e+09 5.04748843e+09
 5.04748846e+09 5.04748849e+09]
(198, 16384, 4) (198, 16384, 4)
18 198


In [32]:
#check selection of baselines/antenna data
ant=10
taql_command=("SELECT TIME,ANTENNA1, ANTENNA2, "
              "[select NAME from ::ANTENNA][ANTENNA1] as ANTNAME1, "
              "[select NAME from ::ANTENNA][ANTENNA2] as ANTNAME2 "
              "from {0} WHERE ANTENNA1!=ANTENNA2 && (ANTENNA1={1} || ANTENNA2={1})").format(
    msfile,ant)
t=pt.taql(taql_command)
t

TIME,ANTENNA1,ANTENNA2,ANTNAME1,ANTNAME2
28-Oct-2018/23:59:31,0,10,RT2,RTC
28-Oct-2018/23:59:31,1,10,RT3,RTC
28-Oct-2018/23:59:31,2,10,RT4,RTC
28-Oct-2018/23:59:31,3,10,RT5,RTC
28-Oct-2018/23:59:31,4,10,RT6,RTC
28-Oct-2018/23:59:31,5,10,RT7,RTC
28-Oct-2018/23:59:31,6,10,RT8,RTC
28-Oct-2018/23:59:31,7,10,RT9,RTC
28-Oct-2018/23:59:31,8,10,RTA,RTC
28-Oct-2018/23:59:31,9,10,RTB,RTC


In [55]:
#okay, so my antenna selection now works! (see above)
#so I have visibilities per antenna in vis_corrected_ant
#so now i just need to do proper averages for each pol
#i want to average over first dimension
print vis_corrected_ant[0].shape
vis_corrected_avg = np.empty(len(ant_names),dtype=object)
for ant in xrange(len(ant_names)):
    vis_corrected_avg[ant] = np.mean(vis_corrected_ant[ant],axis=0)
    
    
print vis_corrected_avg[0].shape

(198, 16384, 4)
(16384, 4)


In [ ]:
#so that should be what I want
#now I can just plot vs frequency, pulling proper stokes param,
#and using np.angle or np.abs

In [69]:
#also got some commands from Tammo Jan that can help clean this up
#so let's test those and see if I get what I expect
taql_antnames = "SELECT NAME FROM {0}::ANTENNA".format(msfile)
t= pt.taql(taql_antnames)
ant_names=t.getcol("NAME")
#print ant_names

taql_freq = "SELECT CHAN_FREQ FROM {0}::SPECTRAL_WINDOW".format(msfile)
t = pt.taql(taql_freq)
freqs = t.getcol('CHAN_FREQ')

vis_corrected_ant = np.empty(len(ant_names),dtype=object)
for ant in xrange(len(ant_names)):
    taql_command = ("SELECT means(CORRECTED_DATA,0) "
                    "as corrected"
                    " FROM {0} " 
                    "WHERE ANTENNA1!=ANTENNA2 && "
                    "(ANTENNA1={1} || ANTENNA2={1})").format(msfile,ant)
    t = pt.taql(taql_command)
    vis_corrected_ant[ant] = t.getcol('corrected')
#    times = t.getcol('TIME')
    
taql_time =  "select TIME from {0} orderby unique TIME".format(msfile)
t= pt.taql(taql_time)
times = t.getcol('TIME')  

print vis_corrected_ant[0].shape

(198, 4)


In [ ]:
#not getting the shape I expect - 
#averaging over frequency instead of what I want

In [70]:
#check shape of data returned from BP tables
#want to use this as a guide for formatting arrays for gains
taql_command = "SELECT TIME,abs(CPARAM) AS amp, arg(CPARAM) AS phase FROM {0}".format(bptable)
t=pt.taql(taql_command)
times = t.getcol('TIME')
amp_sols=t.getcol('amp')
phase_sols = t.getcol('phase')

print amp_sols.shape
#okay, shape is n_ant,n_chan,n_stokes

#in comparison, sahpe for gains (select antenna manually)
#is ntime, 1, nstokes

(12, 16384, 2)


In [71]:
t_field = pt.table(aptable+"::FIELD", readonly=True)
t_field

Successful readonly open of default-locked table /data/adams/apertif/crosscal/181026063/00/raw/WSRTA181026063_B000.G1ap::FIELD: 9 columns, 1 rows


DELAY_DIR,PHASE_DIR,REFERENCE_DIR,CODE,FLAG_ROW,NAME,NUM_POLY,SOURCE_ID,TIME
"[05h42m36.138, +049d51m07.232]","[05h42m36.138, +049d51m07.232]","[05h42m36.138, +049d51m07.232]",3C147_0 BEAM-00,False,3C147,0,-1,28-Oct-2018/23:59:16


In [78]:
taql_stokes = "SELECT abs(CPARAM) AS amp from {0} limit 1" .format(aptable)
t_pol = pt.taql(taql_stokes)
pol_array = t_pol.getcol('amp')
npol = pol_array.shape[2]
print npol


2


In [3]:
#check if get_gain_sols has expected behavior
ant_names,times,amp_ant_array,phase_ant_array = cc.get_gain_sols(aptable)

print amp_ant_array.shape

NameError: name 'cc' is not defined